In [1]:
import pandas as pd 
import numpy as np

TWEET_DATA = pd.read_csv("dataset_final_fixing.csv", encoding = "ISO-8859-1")

TWEET_DATA.head()

,tweet,label
0,@LisaAmartatara3 @fadlizon Yg dibangga banggai...,1
1,RT @RustamIbrahim: POLLING: Dari 4 gubernur di...,1
2,ojok ampe Anies Baswedan menikmati hsil proyek...,1
3,RT @ghanieierfan: Pertemuan sangat mengharukan...,0
4,RT @SebastianTeza: jangn nympek Anies Baswedan...,1


In [3]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].str.lower()


print('Case Folding Result : \n')
print(TWEET_DATA['tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    @lisaamartatara3 @fadlizon yg dibangga banggai...
1    rt @rustamibrahim: polling: dari 4 gubernur di...
2    ojok ampe anies baswedan menikmati hsil proyek...
3    rt @ghanieierfan: pertemuan sangat mengharukan...
4    rt @sebastianteza: jangn nympek anies baswedan...
Name: tweet, dtype: object






In [6]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace("\t","").replace("\n","").replace("\\","")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'ignore').decode('ascii')
    # remove mention, link, hashtag only
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", text).split())
                
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['tweet'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(TWEET_DATA['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [yg, dibangga, banggain, sama, buzzer, cendana...
1    [rt, polling, dari, gubernur, di, jawa, ridwan...
2    [ojok, ampe, anies, baswedan, menikmati, hsil,...
3    [rt, pertemuan, sangat, mengharukan, dgn, ny, ...
4    [rt, jangn, nympek, anies, baswedan, menikmati...
Name: tweet_tokens, dtype: object






In [7]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

TWEET_DATA['tweet_tokens_fdist'] = TWEET_DATA['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(TWEET_DATA['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(yg, 1), (dibangga, 1), (banggain, 1), (sama,...
1    [(rt, 1), (polling, 1), (dari, 1), (gubernur, ...
2    [(ojok, 1), (ampe, 1), (anies, 1), (baswedan, ...
3    [(rt, 1), (pertemuan, 1), (sangat, 1), (mengha...
4    [(rt, 1), (jangn, 1), (nympek, 1), (anies, 1),...
Name: tweet_tokens_fdist, dtype: object


In [8]:
from nltk.corpus import stopwords

# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'ya', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt'])

list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if not word in list_stopwords]

TWEET_DATA['tweet_tokens_WSW'] = TWEET_DATA['tweet_tokens'].apply(stopwords_removal) 


print(TWEET_DATA['tweet_tokens_WSW'].head())

0    [dibangga, banggain, buzzer, cendana, yah, cum...
1    [polling, gubernur, jawa, ridwan, kamil, anies...
2    [ojok, ampe, anies, baswedan, menikmati, hsil,...
3    [pertemuan, mengharukan, tsuneo, yamada, orang...
4    [jangn, nympek, anies, baswedan, menikmati, hs...
Name: tweet_tokens_WSW, dtype: object


# Normalisasi

In [9]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

TWEET_DATA['tweet_normalized'] = TWEET_DATA['tweet_tokens_WSW'].apply(normalized_term)

TWEET_DATA['tweet_normalized'].head(10)

0    [dibangga, banggakan, buzzer, cendana, ya, cum...
1    [polling, gubernur, jawa, ridwan, kamil, anies...
2    [jangan, sampai, anies, baswedan, menikmati, h...
3    [pertemuan, mengharukan, tsuneo, yamada, orang...
4    [jangan, nympek, anies, baswedan, menikmati, h...
5    [mencapai, anies, baswedan, menikmati, produk,...
6    [niat, cari, prestasi, ilegal, anies, baswedan...
7    [berbatas, anies, baswedan, menikmati, rakitan...
8    [jangan, sampai, anies, baswedan, menikmati, h...
9    [jangan, sampai, anies, baswedan, menikmati, h...
Name: tweet_normalized, dtype: object

In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in TWEET_DATA['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

TWEET_DATA['tweet_tokens_stemmed'] = TWEET_DATA['tweet_normalized'].swifter.apply(get_stemmed_term)
print(TWEET_DATA['tweet_tokens_stemmed'])

57313
------------------------
dibangga : bangga
banggakan : bangga
buzzer : buzzer
cendana : cendana
ya : ya
cuman : cuman
orangn : orangn
almarhum : almarhum
soeharton : soeharton
anies : anies
baswedannyg : baswedannyg
polling : polling
gubernur : gubernur
jawa : jawa
ridwan : ridwan
kamil : kamil
baswedan : baswedan
ganjar : ganjar
pranowo : pranowo
khofifah : khofifah
indar : indar
parawansa : parawansa
penu : penu
jangan : jangan
sampai : sampai
menikmati : nikmat
hasil : hasil
proyek : proyek
monas : monas
yang : yang
penuh : penuh
kejanggalannya : janggal
usutskandalmonasgate : usutskandalmonasgate
pertemuan : temu
mengharukan : haru
tsuneo : tsuneo
yamada : yamada
orang : orang
tua : tua
angkat : angkat
goodbener : goodbener
kuliah : kuliah
sophia : sophia
univeu : univeu
nympek : nympek
mencapai : capai
produk : produk
cetak : cetak
biru : biru
monumen : monumen
nasional : nasional
tumpat : tumpat
usutskandalmou : usutskandalmou
niat : niat
cari : cari
prestasi : prestasi
ile

bondowoso : bondowoso
gerak : gerak
cepat : cepat
bantu : bantu
warga : warga
terdampak : dampak
bandang : bandang
ijen : ijen
update : update
pembuatan : buat
rumah : rumah
huntara : huntara
korban : korban
relawan : rawan
sigap : sigap
kp : kp
cigobang : cigobang
lebak : lebak
banten : banten
landa : landa
penajam : najam
paser : paser
utara : utara
sekolah : sekolah
diliburkan : libur
bpbd : bpbd
kesulitan : sulit
bersihkan : sih
material : material
blitar : blitar
com : com
netizen : netizen
udeuddudeuddudeuddnntim : udeuddudeuddudeuddnntim
hore : hore
tukang : tukang
puji : puji
tepuk : tepuk
tangan : tangan
memuji : puji
gabener : gabener
tanggung : tanggung
daya : daya
pikir : pikir
melampaui : lampau
berpolitik : politik
anus : anus
orgu : orgu
beritanya : berita
silaturahmi : silaturahmi
desa : desa
pdtt : pdtt
gus : gus
wamen : wamen
slrh : slrh
pegawai : pegawai
kemendes : kemendes
rangkanpenguatan : rangkanpenguatan
reformasi : reformasi
rakhman : rakhman
tpp : tpp
sumsel :

sejagatnusunda : sejagatnusunda
empirenuvirus : empirenuvirus
coronannmana : coronannmana
top : top
viral : viral
udeudd : udeudd
sukacita : sukacita
merayakan : raya
natal : natal
lupa : lupa
berempati : empat
mengalami : alami
alam : alam
sahabatbagisemua : sahabatbagisemua
natalparlemen : natalparlemen
padau : padau
indonesiaud : indonesiaud
mercy : mercy
komplek : komplek
parlemen : parlemen
senayan : senayan
selasa : selasa
gulai : gulai
trakhir : trakhir
memang : memang
kiriman : kirim
mane : mane
katulampau : katulampau
kerap : kerap
cocok : cocok
skema : skema
rangka : rangka
kegiatan : giat
gunadarma : gunadarma
mengadakan : ada
penggalangan : galang
bahan : bahan
baju : baju
tren : tren
muncul : muncul
empire : empire
bppt : bppt
tmc : tmc
pelaksanaan : laksana
bekerjasama : bekerjasama
tniau : tniau
tindak : tindak
lanjutu : lanjutu
mugo : mugo
ra : ra
banjiruddudef : banjiruddudef
dunia : dunia
huru : huru
hara : hara
nkonflik : nkonflik
usa : usa
irannbanjir : irannbanjir


basuki : basuki
keras : keras
menguras : uras
underpass : underpass
kemayoran : mayor
nnsaat : nnsaat
gabeneru : gabeneru
kemarin : kemarin
jam : jam
bilangu : bilangu
polisi : polisi
india : india
uddudenn : uddudenn
surut : surut
pencitrau : pencitrau
pelaksaan : laksa
masalahgiliran : masalahgiliran
jabat : jabat
ribet : ribet
sekda : sekda
nnnon : nnnon
jobkan : jobkan
tuman : tuman
nrevitalisasi : nrevitalisasi
izinemang : izinemang
e : e
mbah : mbah
ketahuan : tahu
nera : nera
pejabat : jabat
kbingungan : kbingungan
nnmakanya : nnmakanya
dprd : dprd
action : action
dong : dong
belahan : bahan
urus : urus
izin : izin
ndan : ndan
ngantongin : ngantongin
herannya : heran
tindakan : tindak
nsekuat : nsekuat
sehebat : hebat
nieslayakdimakzulkan : nieslayakdimakzulkan
bahlul : bahlul
mengelabui : kelabu
bisa : bisa
nieshancurkanmonas : nieshancurkanmonas
nakula : nakula
dosademokrat : dosademokrat
dosapdip : dosapdip
dosa : dosa
jadin : jadin
polisikanniesn : polisikanniesn
polisikanni

kawann : kawann
anieskamabinghitamn : anieskamabinghitamn
rewoo : rewoo
disehatkan : sehat
lubang : lubang
biofori : biofori
menurutnya : turut
sungai : sungai
miliar : miliar
ancuuurrrrr : ancuuurrrrr
copot : copot
berhentikan : henti
pjlp : pjlp
dishub : dishub
merangkap : rangkap
aniesdiserangbuzzeristana : aniesdiserangbuzzeristana
togog : togog
bacin : bacin
maninuddudcuddudcduddudcan : maninuddudcuddudcduddudcan
upaya : upaya
mengecilkan : kecil
republik : republik
sungguh : sungguh
mengerikan : keri
jeratan : jerat
syetan : syetan
terkutuk : kutuk
akek : akek
beretorika : retorika
sajahasilnya : sajahasilnya
gugatniesgantirugi : gugatniesgantirugi
berbahagialah : bahagia
nnjika : nnjika
bergembiralahnnjika : bergembiralahnnjika
bersyukurlah : syukur
nnkarena : nnkarena
menghargai : harga
perbedaan : beda
mencintai : cinta
nsemuanya : nsemuanya
diayomi : ayom
anieskerjanyata : anieskerjanyata
nnbahagia : nnbahagia
nnmeriang : nnmeriang
hatters : hatters
aniespresiden : aniespresi

...

follor : follor
kamus : kamus
unfolloe : unfolloe
kembalinnmatur : kembalinnmatur
suwun : suwun
akngmas : akngmas
afdhol : afdhol
pencerahannya : cerah
pencinta : cinta
serbuu : serbuu
uddudcduddudcaa : uddudcduddudcaa
keukeuh : keukeuh
meningan : ning
fahamnya : fahamnya
dongen : dongen
bangkitnya : bangkit
berkeadilan : adil
isme : isme
sampyan : sampyan
menuduh : tuduh
mengedit : edit
harma : harma
mate : mate
pasi : pasi
nteman : nteman
berminat : minat
ketiknyes : ketiknyes
saranulove : saranulove
suspend : suspend
cadangu : cadangu
pamen : pamen
diremehkan : remeh
buzerp : buzerp
yanjuga : yanjuga
beeeeee : beeeeee
teguh : teguh
nbhinneka : nbhinneka
ampnnkri : ampnnkri
metonk : metonk
nperiode : nperiode
mesjid : mesjid
mencu : mencu
pendaptaran : pendaptaran
wibnnsyarat : wibnnsyarat
nsiap : nsiap
sesi : sesi
uddudcfnnnkri : uddudcfnnnkri
dibubarin : dibubarin
sebarisan : baris
hadeeeh : hadeeeh
wet : wet
botol : botol
dipiara : piara
lattee : lattee
ryfae : ryfae
nnnkri : nnnk

sutransupport : sutransupport
tidurnyau : tidurnyau
vizard : vizard
modusnya : modus
kbhinekaan : kbhinekaan
nkriudcuddeeudcudde : nkriudcuddeeudcudde
batch : batch
daring : daring
barengan : bareng
penjagu : penjagu
pendaftarann : pendaftarann
sutran : sutran
bokebnakun : bokebnakun
nkrinntinggalkan : nkrinntinggalkan
akunu : akunu
kitaid : kitaid
stroke : stroke
uddudefudcudffd : uddudefudcudffd
perpecahan : pecah
kuisjatengkita : kuisjatengkita
hayooi : hayooi
ramekan : ramekan
programfolbacknretweet : programfolbacknretweet
nsupport : nsupport
nuddudeab : nuddudeab
kadrunnuddudeab : kadrunnuddudeab
bokepnu : bokepnu
nkriuddudc : nkriuddudc
merapatkan : rapat
abubakarn : abubakarn
gegayaan : gegayaan
keris : keris
favoritku : favorit
forkopimcam : forkopimcam
purwodadi : purwodadi
istigosah : istigosah
mwc : mwc
purwodadinnpurwodadi : purwodadinnpurwodadi
kubro : kubro
diadakan : ada
difollow : difollow
nterbang : nterbang
uufefuufefuufefn : uufefuufefuufefn
yahhn : yahhn
uddudeuddu

lahudeudd : lahudeudd
nsurat : nsurat
ugm : ugm
wooiiinini : wooiiinini
npaedofil : npaedofil
karmann : karmann
superior : superior
cuihhhh : cuihhhh
jr : jr
luu : luu
mat : mat
nkrinpagi : nkrinpagi
pecundang : cundang
blokir : blokir
begitupula : begitupula
neu : neu
tunjukkan : tunjuk
taringnnindonesia : taringnnindonesia
kitannyatakan : kitannyatakan
nntegas : nntegas
norma : norma
dipatuhi : patuh
lemahu : lemahu
kembung : kembung
nnkomentaru : nnkomentaru
akarnya : akar
brader : brader
uanh : uanh
tatkala : tatkala
berkau : kau
yuuu : yuuu
sedulur : dulur
putung : putung
nrawe : nrawe
rawe : rawe
rantas : rantas
dokma : dokma
sesat : sesat
zealand : zealand
irlandia : irlandia
politikal : politikal
menjajah : jajah
pacarmu : pacar
ditaksir : taksir
direbut : rebut
twitters : twitters
npagi : npagi
badminton : badminton
pemanasan : panas
luncurin : luncurin
lumu : lumu
uddudcdntak : uddudcdntak
mengabdi : abdi
polisiindonesia : polisiindonesia
beritapolisin : beritapolisin
polisis

dahsyaaaaat : dahsyaaaaat
siapnuntuk : siapnuntuk
mengamankan : aman
nkrindari : nkrindari
invasi : invasi
selengkapnyu : selengkapnyu
npencinta : npencinta
matinnkri : matinnkri
basin : basin
jokowikawalnatunanjokowi : jokowikawalnatunanjokowi
viva : viva
kimxena : kimxena
satriadhi : satriadhi
folback : folback
soemantri : soemantri
dianudin : dianudin
veraagustin : veraagustin
djaswoko : djaswoko
sora : sora
baltasar : baltasar
yunan : yunan
promotorn : promotorn
uddudeudduded : uddudeudduded
kepri : kepri
dkknnreflay : dkknnreflay
daftarnnsyarat : daftarnnsyarat
fbu : fbu
illahi : illahi
toea : toea
cintah : cintah
kadrunuddudeuddudeudeudduddudeudeudd : kadrunuddudeuddudeudeudduddudeudeudd
uddudcduddudcduddudcduddudcduddudcduddudcduddudcd : uddudcduddudcduddudcduddudcduddudcduddudcduddudcd
natunannatuna : natunannatuna
nkrinnkri : nkrinnkri
matinkami : matinkami
selalunsupport : selalunsupport
kedaulau : kedaulau
nnokadrun : nnokadrun
nobokepnkomen : nobokepnkomen
indonesiadamaiu :

nserentakin : nserentakin
pilpresjiwasrayannama : pilpresjiwasrayannama
kecebong lah : kecebong lah
allahbukti : allahbukti
wisudafoto : wisudafoto
fuji : fuji
uddudebnn : uddudebnn
uddudenjangan : uddudenjangan
kegiatanku : giat
input : input
duana : duana
eta : eta
kinn : kinn
ndoel : ndoel
rusakk : rusakk
nnemohh : nnemohh
guyon : guyon
nginikiuddudeudduddnn : nginikiuddudeudduddnn
dvina : dvina
seberani : berani
sekuat : kuat
jantungmu : jantung
udeudddnnklu : udeudddnnklu
nyerah : nyerah
uddudecnn : uddudecnn
pilpresjiwasrayau : pilpresjiwasrayau
kabina : kabina
binan : bin
hwaduh : hwaduh
seehh : seehh
okehub : okehub
ajjaaa : ajjaaa
maekan : maekan
lagee : lagee
bijimanapun : bijimanapun
tunggoo : tunggoo
ajjahhhuddude : ajjahhhuddude
eonya : eonya
kpunnkpu : kpunnkpu
singkatan : singkat
komisu : komisu
woow : woow
tali : tali
nhnn : nhnn
bungkusudduden : bungkusudduden
cabok : cabok
bulak : bulak
aya : aya
kabarnann : kabarnann
hushhra : hushhra
wani : wani
nyebut : nyebut
hebe

In [10]:
TWEET_DATA.to_csv("Text_Preprocessing.csv")

In [11]:
TWEET_DATA.to_excel("Text_Preprocessing.xlsx")

In [12]:
TWEET_DATA.to_hdf("Text_Preprocessing.h5")

TypeError: to_hdf() missing 1 required positional argument: 'key'